Code for scraping from indeed.co.uk

In [108]:
#import packages
import requests, bs4, time
import pandas as pd
import os
from itertools import cycle
import traceback
from datetime import date
import pickle
import numpy as np
import datetime
import cleaning_funcs_indeed_uk as cf


In [110]:
# readin all raw pickle files

ads= pd.DataFrame(columns = ['company', 'job_title', 'salary', 'location', 'description', 'date',
       'full_description', 'other_deets', 'jobtype', 'salary_low',
       'salary_high','extraction_date'])

import os

for i in os.listdir('C:/Users/lundr/DataScienceJobs/data/indeed_raw_pickles/'):
    
    read_in = pd.read_pickle("C:/Users/lundr/DataScienceJobs/data/indeed_raw_pickles/"+i)
    ads = pd.concat([ads,read_in],ignore_index = True, axis = 0, sort=True)


In [131]:
#drop rows with identical descriptions
ads = ads.drop_duplicates(['full_description', 'company'])

#reset index after dropping duplicated
ads.reset_index(inplace=True)

In [113]:
# clean data and create additional variables

ads['location'],ads['salary'],ads['jobtype'] = cf.clean_other_deets(ads['other_deets'])
ads['full_description'] = cf.clean_column(ads['full_description'])
ads['company'] = cf.clean_column(ads['company'])
ads['salary_low'], ads['salary_high'],ads['salary_type'] = cf.create_split_salary_range(ads['salary'])
ads['salary_low']=cf.clean_salary(ads['salary_low'],"£")
ads['salary_high']=cf.clean_salary(ads['salary_high'],"£")
ads['days_from_post'] = cf.create_days_from_post(ads['date'])
ads['posted_date'] = cf.create_posted_date(ads['days_from_post'],ads['extraction_date'])



In [153]:
# check for salary anomalies and manually correct if needed
idx = ads.loc[ads['salary_high'] != 'None']
idx[idx['salary_high']/idx['salary_low']>3]
#

,index,company,date,extraction_date,full_description,job_title,jobtype,location,salary,salary_high,...,salary_type,salary_average,days_from_post,posted_date,industry,education,career,country,ref_code,Region
2053,5259,Capita IT Resourcing,18 days ago,6-11-2019,Data Analysts - month initial contract Based...,Data Analyst,Contract,Sheffield,"£100 - £1,000 a day",1000,...,day,550,18,2019-05-24 00:00:00,NaN,NaN,NaN,UK,NaN,Yorkshire and The Humber
3413,9219,Cyber Tech Company,Nothing_found,11-11-2019,Sales Executive (Cyber Tech Company)London W ...,Sales Executive,"Full-time, Permanent",London,"£30,000 - £100,000 a year",100000,...,year,65000,None,None,NaN,NaN,NaN,UK,NaN,London


In [154]:
#generate salary average after correction
ads['salary_average'] = cf.salary_average(ads['salary_low'],ads['salary_high'])

In [116]:
# generate missing columns

ads = ads.drop(['description', 'other_deets'], axis=1)
#ads.columns = ['company', 'job_title',  'salary','location','duration', 'description', 'jobtype','salary_low', 'salary_high']


#industry: which industry sector
    
ads['industry'] = np.nan

#education: required level and subject of education

ads['education'] = np.nan

#**career: whether entry level, experienced etc
ads['career'] = np.nan

# country it relates to

ads['country'] = "UK"

# job refcode
ads['ref_code'] = np.nan


In [135]:
# Add regional coluns
# read in regional lookup
loc = pd.read_csv('~/DataScienceJobs/data/uk_location_lookup.csv')

In [136]:
# cycle through lookup list

import re

a = list(np.zeros(len(ads)))

for j in range(len(ads['location'])):
    if type(ads['location'][j]) == float:
        pass
    else:
        for i in range(len(loc)):
            x = re.search(loc['LA'][i],ads['location'][j])
            y = re.search(ads['location'][j],loc['LA'][i])
            z = re.search(ads['location'][j],loc['Region'][i])
            w = re.search(loc['Region'][i], ads['location'][j])
            if (x) or (y) or (z) or (w):
                a[j] = loc['Region'][i]
            else:
                pass
                

ads['Region'] = a       

In [137]:
# check for any which did not get allocated a region
ads.loc[ads['Region']==0, 'location'].unique()

array([nan, "St. Mary's"], dtype=object)

In [134]:
#find rows with no salaries
ads[ads['salary_low']!='None']

,index,company,date,extraction_date,full_description,job_title,jobtype,location,salary,salary_high,...,salary_type,salary_average,days_from_post,posted_date,industry,education,career,country,ref_code,Region
0,0,Secret Intelligence Service,Nothing_found,2019-11-13,Department: Intelligence Location(s): London S...,Intelligence Officer,None,London,"£33,800 - £42,700 a year",42700,...,year,38250,None,None,NaN,NaN,NaN,UK,NaN,London
1,1,Norfolk County Council,Just posted,2019-11-13,hours per week( hours per week and hou...,Business and Engagement Officers,"Full-time, Part-time, Permanent",Norwich,"£29,636 - £31,371 a year",31371,...,year,30503,0,2019-11-13 00:00:00,NaN,NaN,NaN,UK,NaN,East of England
3,3,Com,Nothing_found,2019-11-13,Help us shape the future of telecoms as a Busi...,Business Intelligence Analyst,None,Bournemouth,"£35,000 - £50,000 a year",50000,...,year,42500,None,None,NaN,NaN,NaN,UK,NaN,South West
4,4,Secret Intelligence Service,30+ days ago,2019-11-13,Intelligence is at the heart of what MI does ...,Intelligence Officer,None,London,"£33,800 - £42,700 a year",42700,...,year,38250,30,2019-10-14 00:00:00,NaN,NaN,NaN,UK,NaN,London
5,5,Honeycomb Group,19 days ago,2019-11-13,Honeycomb Group is a team of social-minded bra...,Business Intelligence Assistant,None,Stoke-on-Trent ST4,"£26,250 a year",26250,...,year,26250,19,2019-10-25 00:00:00,NaN,NaN,NaN,UK,NaN,West Midlands
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4572,12848,Huckletree,11 days ago,11-11-2019,It’s important to note that this internship is...,Community Interns,None,Shoreditch,"£19,000 - £21,000 a year",21000,...,year,20000,11,2019-10-31 00:00:00,NaN,NaN,NaN,UK,NaN,London
4573,12849,Futureheads Recruitment,16 days ago,11-11-2019,Futureheads is currently working with an excit...,Director of AI,Permanent,London,"£80,000 - £90,000 a year",90000,...,year,85000,16,2019-10-26 00:00:00,NaN,NaN,NaN,UK,NaN,London
4574,12851,Arctic Shores,30+ days ago,11-11-2019,What we re after and how you d fit in: We re l...,Python Software Engineer (Contract),None,Manchester,£450 a day,450,...,day,450,30,2019-10-12 00:00:00,NaN,NaN,NaN,UK,NaN,North West
4582,12887,University of Oxford,30+ days ago,11-11-2019,Department of Computer Science Parks Road Ox...,Research Associate on FUN2MODEL,None,Oxford,"£32,236 - £39,609 a year",39609,...,year,35922,30,2019-10-12 00:00:00,NaN,NaN,NaN,UK,NaN,South East
